# Testing OpenSmile Classes

Draft 1 of pipeline for creating a df, loading the generated openSMILE features, and running feature selection.

In [34]:
import pandas as pd
import numpy as np
import sys
import os
import json
from tqdm import tqdm
sys.path.append("/home/ubuntu/MultiModalDeepFake")
from packages.LJDataLoader import LJDataLoader
from packages.SmileFeatureLoader import smileFeatureLoader
from packages.SmileFeatureSelector import smileFeatureSelectorBruteForce, smileFeatureSelectFromModel, smileUnivariateFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.feature_selection as fs

## Part 1: Creating the dataframe with the train-dev-test split

In [2]:
#load LJ 16khz data
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)

In [3]:
#split data into train, dev, test
loader.splitData()

13055
7833 10444


In [4]:
loader.metadata.columns

Index(['index', 'Unnamed: 0', 'id', 'Main_ID', 'Secondary_ID', 'Real',
       'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge',
       'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow', 'ElevenLabs',
       'UberDuck', 'transcript_1', 'transcript_2', 'type'],
      dtype='object')

In [5]:
#get the column names
cols = list(loader.metadata.columns[5:15])
cols

['Real',
 'Full_Band_MelGan',
 'HifiGan',
 'MelGan',
 'MelGanLarge',
 'Multi_Band_MelGan',
 'Parallel_WaveGan',
 'Waveglow',
 'ElevenLabs',
 'UberDuck']

In [6]:
#generate the dataframe
#single_id entry=True breaks
df = loader.generateFinalDataFrame(real_col=cols[0], fake_cols=cols[1:])

In [7]:
df.shape

(130550, 6)

In [8]:
df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ010-0214,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ014-0253,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ005-0168,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ011-0264,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ017-0067,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [9]:
# sample 10000 rows from df for testing with balanced real and fake
#sampled_df = df.groupby('B', group_keys=False).apply(lambda x: x.sample(n=min_group_size))

#temporary solution for testing

test_df = df.groupby('label', group_keys=False).apply(lambda x: x.sample(n=500))

In [10]:
#check balance
test_df.type.value_counts()

train    601
test     202
dev      197
Name: type, dtype: int64

In [11]:
test_df.architecture.value_counts()

Real                 500
Multi_Band_MelGan     67
MelGanLarge           65
ElevenLabs            59
Waveglow              58
UberDuck              57
MelGan                55
Full_Band_MelGan      51
HifiGan               51
Parallel_WaveGan      37
Name: architecture, dtype: int64

In [12]:
test_df.label.value_counts()

0    500
1    500
Name: label, dtype: int64

## Side: Creating/Updating File Mappings JSON:

This JSON has the path to the collection of wavfiles as the key and the corresponding smile features as values. 

For now, each time a new set of wav file are generated for a new provider/architecture - they should be stored in a separate unique folderpath. The `SmileFeatureGenerator` class will convert these to the csv file that can then be used to create a new JSON entry. 

In [13]:
file_mapping = dict()

#biden real
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/44KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/16KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv'

#biden eleven labs
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/44Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/16Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-16Khz.csv'

#biden in the wild
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-Original.csv'
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

#ljspeech real
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-16000KHz.csv'

#ljspeech eleven labs
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-16000KHz.csv'

#ljspeech uberduck
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv'

#ljspeech melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-16000KHz.csv'

#ljspeech melgan large
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-16000KHz.csv'

#ljspeech multi band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-16000KHz.csv'

#ljspeech full band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-16000KHz.csv'

#ljspeech hifiGAN
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-16000KHz.csv'

#ljspeech parallel wavegan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-16000KHz.csv'

#ljspeech waveglow
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-16000KHz.csv'

#file_mapping

In [14]:
cellEnabled=0

if cellEnabled:

    file_mappings = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/' + 'file_mappings.json' 

    with open(file_mappings, "w") as json_file:
        json.dump(file_mapping, json_file, indent=4)

## Part 2: Loading the openSMILE features

The openSMILE features are generated and stored in multiple `csv` files here - `data/OpenSmile/04-2023-OSF-feature-csvs`. This folder also stores the JSON file

In [15]:
#pass the test df and the smileFeatureLoader object does the merging
smFeatLoader = smileFeatureLoader(test_df)

100%|██████████| 10/10 [02:02<00:00, 12.21s/it]


In [16]:
#get the df for feature selection
fs_df = smFeatLoader.return_merged_df()

In [17]:
fs_df.shape

(1000, 6380)

In [18]:
fs_df.head()

,type,id,architecture,path,label,multiclass_label,duration(seconds),audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,test,LJ013-0259,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.302,1.496595,0.042553,0.468085,...,7.460823,0.592091,2.315921,2.339794,-19.323751,0.551804,105.872765,61.772340,113.69934,64.144070
1,dev,LJ012-0263,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.651,1.563841,0.768559,0.578603,...,9.612248,0.600741,3.082802,3.058858,19.632143,0.509521,153.061510,64.131256,144.06512,78.137570
2,train,LJ028-0354,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,8.366,2.199751,0.375151,0.322075,...,9.524864,0.578718,2.724679,2.711402,19.890615,0.501133,127.369700,67.013660,119.74008,60.782593
3,train,LJ016-0167,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.511,1.935269,0.022523,0.184685,...,7.462513,0.437238,2.635641,2.659336,-19.490585,0.571664,123.250435,77.503330,106.04592,61.903313
4,train,LJ039-0039,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,5.870,2.389787,0.267241,0.251724,...,8.003578,0.532049,2.905448,2.901232,19.999120,0.460577,133.209290,59.059555,121.85595,56.025913


## Part 3: Running feature selection

In [19]:
#get the columns that are not the 6373 features as metadata
metadata = list(fs_df.columns[:7])
metadata

['type',
 'id',
 'architecture',
 'path',
 'label',
 'multiclass_label',
 'duration(seconds)']

### BruteForce feature selection (Buggy and doesn't run):

Has an issue due to how architecture is currently set up. Need a mapping between real and fake architectures for the code to run properly. Gautham to discuss further with Romit.

In [20]:
#initialize the object
smFS_bf = smileFeatureSelectorBruteForce(df=fs_df, metadata=metadata, standardize=True, model=LogisticRegression())

Initializing data...
Standardizing features...
smileFeatureSelectorBruteForce object initialized.



__architecture column needs fixing (real should be mapped to the corresponding fake that is assigned to it__

In [24]:
smFS_bf.data.architecture.unique()

array(['Real', 'MelGan', 'ElevenLabs', 'UberDuck', 'MelGanLarge',
       'Full_Band_MelGan', 'HifiGan', 'Waveglow', 'Multi_Band_MelGan',
       'Parallel_WaveGan'], dtype=object)

In [26]:
smFS_bf.data.architecture.value_counts()

Real                 500
Multi_Band_MelGan     67
MelGanLarge           65
ElevenLabs            59
Waveglow              58
UberDuck              57
MelGan                55
Full_Band_MelGan      51
HifiGan               51
Parallel_WaveGan      37
Name: architecture, dtype: int64

### Select From Model

In [30]:
smFS_model = smileFeatureSelectFromModel(df=fs_df,
                                        metadata=['type', 'id', 'architecture', 'path', 'label', 'multiclass_label', 'duration(seconds)'],
                                        standardize = True,
                                        model=RandomForestClassifier())

Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



In [31]:
exp_df_2 = smFS_model.select_features(max_features=50, arch='all_archs', return_df=True)


Top 50 features selected from all_archs architecture.

1. audspec_lengthL1norm_sma_minPos
2. audspec_lengthL1norm_sma_de_percentile99.0
3. pcm_fftMag_spectralEntropy_sma_range
4. pcm_fftMag_spectralEntropy_sma_stddev
5. pcm_fftMag_spectralEntropy_sma_upleveltime25
6. pcm_fftMag_spectralEntropy_sma_upleveltime75
7. pcm_fftMag_spectralEntropy_sma_lpc2
8. pcm_fftMag_spectralSlope_sma_pctlrange0-1
9. pcm_fftMag_spectralHarmonicity_sma_iqr1-2
10. mfcc_sma[4]_lpc0
11. mfcc_sma[5]_lpc0
12. mfcc_sma[5]_lpc1
13. mfcc_sma[6]_lpc1
14. mfcc_sma[7]_lpgain
15. mfcc_sma[8]_lpgain
16. mfcc_sma[8]_lpc0
17. mfcc_sma[8]_lpc1
18. mfcc_sma[9]_lpgain
19. mfcc_sma[9]_lpc0
20. mfcc_sma[10]_lpgain
21. mfcc_sma[11]_lpc1
22. mfcc_sma[12]_lpgain
23. mfcc_sma[13]_lpgain
24. mfcc_sma[13]_lpc0
25. mfcc_sma[14]_lpgain
26. mfcc_sma[14]_lpc4
27. pcm_fftMag_spectralFlux_sma_de_range
28. pcm_fftMag_spectralEntropy_sma_de_lpc0
29. pcm_fftMag_spectralEntropy_sma_de_lpc2
30. pcm_fftMag_spectralEntropy_sma_de_lpc3
31. pcm_f

In [32]:
exp_df_2.head()

,type,id,architecture,path,label,multiclass_label,duration(seconds),audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_de_percentile99.0,pcm_fftMag_spectralEntropy_sma_range,...,mfcc_sma_de[14]_stddev,jitterLocal_sma_quartile1,jitterLocal_sma_percentile1.0,jitterDDP_sma_flatness,jitterDDP_sma_quartile1,jitterDDP_sma_quartile2,jitterDDP_sma_percentile1.0,shimmerLocal_sma_quartile1,pcm_fftMag_spectralFlux_sma_peakMeanMeanDist,pcm_fftMag_spectralFlux_sma_de_rqmean
0,test,LJ013-0259,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.302,0.468085,0.247302,4.469073,...,1.983607,0.007026,0.001895,0.453075,0.004671,0.008471,0.001323,0.052039,0.218071,0.039611
1,dev,LJ012-0263,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.651,0.578603,0.232772,4.941270,...,2.721441,0.006502,0.001842,0.426867,0.003687,0.008054,0.001291,0.051700,0.275938,0.056681
2,train,LJ028-0354,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,8.366,0.322075,0.283420,4.672210,...,2.294550,0.006055,0.001006,0.373644,0.003025,0.008001,0.000828,0.053240,0.324383,0.058585
3,train,LJ016-0167,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.511,0.184685,0.216809,4.382821,...,2.379282,0.005908,0.001493,0.415245,0.003536,0.008716,0.001126,0.047515,0.331672,0.049816
4,train,LJ039-0039,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,5.870,0.251724,0.359164,4.778096,...,2.391476,0.009301,0.001601,0.368966,0.005124,0.012834,0.001009,0.053185,0.368581,0.072331


### Univariate Feature Selection:

In [33]:
smFS_univariate = smileUnivariateFeatureSelector(df=fs_df,
                                                metadata=['type', 'id', 'architecture', 'path', 'label', 'multiclass_label', 'duration(seconds)'],
                                                standardize = True)

Initializing data...
Standardizing features...
smileUnivariateFeatureSelector object initialized.



In [36]:
exp_df_3 = smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=50, arch='all_archs', return_df=True)


Top 50 features selected from all_archs architecture.

1. audspecRasta_lengthL1norm_sma_lpc3
2. audSpec_Rfilt_sma[6]_iqr1-3
3. pcm_fftMag_fband1000-4000_sma_iqr2-3
4. pcm_fftMag_spectralCentroid_sma_quartile1
5. pcm_fftMag_spectralSlope_sma_upleveltime90
6. mfcc_sma[3]_lpc0
7. mfcc_sma[5]_lpc0
8. mfcc_sma[5]_lpc1
9. mfcc_sma[6]_lpc1
10. mfcc_sma[6]_lpc2
11. mfcc_sma[8]_lpc0
12. mfcc_sma[9]_lpgain
13. mfcc_sma[10]_lpc2
14. mfcc_sma[13]_lpgain
15. audSpec_Rfilt_sma_de[8]_kurtosis
16. pcm_fftMag_spectralEntropy_sma_de_upleveltime25
17. pcm_fftMag_spectralEntropy_sma_de_lpc0
18. pcm_fftMag_spectralEntropy_sma_de_lpc1
19. pcm_fftMag_spectralEntropy_sma_de_lpc2
20. pcm_fftMag_spectralEntropy_sma_de_lpc3
21. pcm_fftMag_spectralEntropy_sma_de_lpc4
22. mfcc_sma_de[3]_leftctime
23. mfcc_sma_de[5]_lpc0
24. mfcc_sma_de[5]_lpc1
25. mfcc_sma_de[5]_lpc2
26. mfcc_sma_de[5]_lpc3
27. mfcc_sma_de[6]_lpc0
28. mfcc_sma_de[6]_lpc1
29. mfcc_sma_de[8]_lpc1
30. mfcc_sma_de[10]_lpc0
31. mfcc_sma_de[11]_percent

In [37]:
exp_df_3.head()

,type,id,architecture,path,label,multiclass_label,duration(seconds),audspecRasta_lengthL1norm_sma_lpc3,audSpec_Rfilt_sma[6]_iqr1-3,pcm_fftMag_fband1000-4000_sma_iqr2-3,...,jitterDDP_sma_percentile1.0,audSpec_Rfilt_sma[11]_stddevRisingSlope,audSpec_Rfilt_sma[17]_linregc1,pcm_fftMag_spectralEntropy_sma_meanPeakDist,pcm_fftMag_spectralEntropy_sma_peakRangeAbs,pcm_fftMag_spectralEntropy_sma_linregerrQ,mfcc_sma[10]_peakMeanRel,audSpec_Rfilt_sma_de[25]_meanRisingSlope,pcm_fftMag_spectralEntropy_sma_de_peakRangeRel,pcm_fftMag_spectralHarmonicity_sma_de_peakMeanRel
0,test,LJ013-0259,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.302,-0.332759,1.157577,0.022394,...,0.001323,27.166035,0.010736,0.181818,2.795163,0.046141,-0.351882,7.344189,0.423907,-20.0
1,dev,LJ012-0263,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.651,-0.247732,1.532883,0.033312,...,0.001291,28.959106,0.006218,0.177083,4.169969,0.063398,-0.626416,7.708104,0.469881,-20.0
2,train,LJ028-0354,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,8.366,-0.438421,1.361229,0.041608,...,0.000828,21.196800,-0.003832,0.174565,3.500376,0.046804,0.140003,8.733562,0.503734,-20.0
3,train,LJ016-0167,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,4.511,-0.171198,1.308590,0.011016,...,0.001126,22.780064,-0.010751,0.177727,3.683311,0.050404,-0.015058,8.424246,0.521879,-20.0
4,train,LJ039-0039,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,5.870,-0.109692,1.320929,0.014708,...,0.001009,19.902828,-0.001488,0.173125,3.361160,0.055270,-5.756375,10.361280,0.420324,-20.0
